In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("attention.pdf")

pdf_docs = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

documents = text_splitter.split_documents(pdf_docs)

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OllamaEmbeddings())

In [4]:
query="An attention function"
result = db.similarity_search(query)
result

[Document(page_content='itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding\nlayers, produce outputs of dimension dmodel = 512 .\nDecoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the two\nsub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head\nattention over the output of the encoder stack. Similar to the encoder, we employ residual connections\naround each of the sub-layers, followed by layer normalization. We also modify the self-attention\nsub-layer in the decoder stack to prevent positions from attending to subsequent positions. This\nmasking, combined with fact that the output embeddings are offset by one position, ensures that the\npredictions for position ican depend only on the known outputs at positions less than i.\n3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,', metadata={

In [5]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful
<context>
{context}
</context>
Question: {input}
""")

In [12]:
# Chain intro
#create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)

In [13]:
#retriever
retriever = db.as_retriever()

In [14]:
#retriever_chain

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [16]:
response = retrieval_chain.invoke({'input':"An attention function"})
response['answer']

'A nice question!\n\nIn the provided context, we can see that the Transformer architecture uses self-attention mechanisms to facilitate residual connections between sub-layers in both the encoder and decoder.\n\nSpecifically, the attention function is used to map a query and a set of key-value pairs to an output. This process allows the model to weigh the importance of different input elements relative to the current processing position.\n\nIn other words, the attention mechanism helps the model focus on relevant parts of the input sequence when making predictions, which is particularly important in machine translation tasks like English-to-German and English-to-French.\n\nSo, to answer your question: An attention function is used to map a query and a set of key-value pairs to an output, allowing the model to weigh the importance of different input elements relative to the current processing position.'